# Heimadæmi 2 - TÖL025M Inngangur að máltækni

Þorvaldur Tumi Baldursson


## 1. Lemmetization


In [5]:
from reynir import Greynir
from collections import Counter
import re

with open("../data/all_stop_words.txt") as f:
    stops = list(f.read().split("\n"))

with open("../data/log.txt") as f:
    data = f.read()[:20000]

f_data = " ".join(filter(lambda w: w not in stops, data.split()))

words = re.sub("(.)([\,\.\:\!\/\?\"\'\„\(\)\…\´])[^.]", r"\1 \2 ", data).split()
words = list(map(lambda w:w.lower(), words))

top_words = sorted(dict(Counter(words)).items(), key=lambda x:x[1])

f_words = re.sub("(.)([\,\.\:\!\/\?\"\'\„\(\)\…\´])[^.]", r"\1 \2 ", f_data).split()
f_words = list(map(lambda w:w.lower(), f_words))

f_top_words = sorted(dict(Counter(f_words)).items(), key=lambda x:x[1])

g = Greynir()
parsed_data = g.parse(data)
f_parsed_data = g.parse(f_data)

### a)

Ég fækkaði fjölda orða í upprunalegu útfærslunni niður í **20000** þar sem ég var að struggla við það að keyra Greyni á allt gagnasettið mitt

Það er alveg slatta munur á milli minnar útfærslu og Greyni, sá greinilegasti er að mín útfærsla grípur töluvert fleiri kommur og punkta auk þess lætur Greynir okkur alveg sleppa við við orðmyndir eins og "vera" og fáum bara "er" í staðinn


In [7]:
from collections import Counter

def all_lemmas(sentences):
    lemmas = []
    if sentences is None:
        return None
    for s in sentences:
        if s.lemmas is not None:
            for l in s.lemmas:
                lemmas.append(l)
    return lemmas

In [3]:
AL = all_lemmas(parsed_data["sentences"])
top_lemmas = list(reversed(sorted(dict(Counter(AL)).items(), key=lambda x: x[1])))
tumi_lemmas = list(reversed(sorted(dict(top_words).items(), key=lambda x: x[1])))

print("Greynir:", dict(top_lemmas[:10]))
print("Tumi:   ", dict(tumi_lemmas[:10]))

Greynir: {'.': 149, ',': 83, 'og': 69, 'ég': 50, 'vera': 50, 'á': 47, 'þú': 44, 'í': 43, 'að': 35, 'ef': 34}
Tumi:    {'.': 283, ',': 280, 'og': 149, 'í': 91, 'á': 84, 'ég': 82, 'að': 76, 'er': 66, 'þú': 46, 'ef': 45}


### b)


In [4]:
FAL = all_lemmas(f_parsed_data["sentences"])
f_top_lemmas = list(reversed(sorted(dict(Counter(FAL)).items(), key=lambda x: x[1])))
f_tumi_lemmas = list(reversed(sorted(dict(f_top_words).items(), key=lambda x: x[1])))

print("Greynir:", dict(f_top_lemmas[:10]))
print("Tumi:   ", dict(f_tumi_lemmas[:10]))

Greynir: {'.': 102, ',': 62, 'vera': 14, 'kæta': 13, 'ef': 12, 'ég': 11, 'átta': 10, 'sofa': 10, 'ei': 8, 'ást': 8}
Tumi:    {'.': 283, ',': 280, 'ja': 30, 'ég': 28, 'la': 25, 'ef': 23, 'kætist': 21, 'áttu': 17, 'í': 17, 'giftist': 16}


### c)


Lemmun hjálpar til við að staðla setningar, ef allar þær setningar sem við vinnum með hafa farið í gegnum lemmun þá endum við með orð á sama formi sem einfaldar ýmis konar *"post process"* vinnu.
Til að nefna dæmi um hluti sem lemmun hjálpar við væri hægt að nefna það að minnka stærð textasafns, ef við höfum bara lemmuðu orðin þá þurfum við ekki að halda utan um allar beygingar og breytingar.
Hinsvegar getur lemmun líka tekið í burtu meiningu ef við erum td. að þýða úr íslensku yfir á ensku.

## 2. Parsing


### a) Sjá efsta code block


### b)


In [5]:
def top_phrases(data, type):
    PS = []
    for sent in [p for p in data if p and p.tree and p.ifd_tags]:
        for np in [p for p in sent.tree.all_matches(type)]:
            PS.append(np.canonical_np)
    return dict(Counter(PS).most_common(10))

In [6]:
print(top_phrases(parsed_data["sentences"], "NP"))

{'þú': 41, 'ég': 38, 'hann': 9, 'það': 8, 'Ég': 8, 'Ást': 6, 'mjólkurbíll': 6, 'glaðværð þín': 5, 'ónei': 4, 'fyrstur': 4}


### c)


In [7]:
print(top_phrases(parsed_data["sentences"], "VP"))

{'er': 38, 'kætist': 18, 'áttu': 18, 'sefur': 15, 'giftist': 12, 'detta': 7, 'var': 6, 'á': 6, 'verður': 5, 'skal': 5}


### d)


In [8]:
ruglandi = [
    "Ási á Á á á í á.", 
    "Galsinna gíraffar í gannisslag.", 
    "Tóti fór og fórst í kórferðinni.", 
    "Áll í ál á lá á ál máli við hliðina á stál skál."
]

print(g.parse_single(ruglandi[0]).tree.lemmas)
print(g.parse_single(ruglandi[1]).tree.lemmas)
print(g.parse_single(ruglandi[2]).tree.lemmas)
print(g.parse_single(ruglandi[3]).tree.lemmas)

['Ási', 'eiga', 'Á', 'á', 'á', 'í', 'á', '.']
['Galsinna', 'gíraffi', 'í', 'gannisslag', '.']
['Tóti', 'fara', 'og', 'fara', 'í', 'kórferð', '.']
['áll', 'í', 'áll', 'á', 'lá', 'á', 'áll', 'mála', 'við', 'hlið', 'á', 'stál', 'skál', '.']


Mér tókst að lokum að rugla Greyni þó ekki mikið, hann er að rugla áli (málminum) við ál (fiskinn? dýrið?) auk þess sem hann heldur að mál (ílát) sé sögnin að mála


## 3 - getting creative


In [10]:
from reynir import Greynir
g = Greynir()

# byrjum á að lemma alla textana sem hægt er að leita í 
search_lemmas = {}
texts = {
    "s1": "Siggi var úti",
    "s2": "Laus og liðuugur",
    "s3": "Snati og Óli",
    "s4": "Sofðu unga ástin mín",
    "s5": "Ég á lítinn skrýtinn skugga",
    "s6": "Komdu kisa mín",
    "s7": "Kvæðið um fuglana",
    "s8": "Krummi krunkar úti",
    "s9": "Til eru fræ",
    "s10": "Lagið um það sem er bannað",
    "s11": "Litla flugan",
    "s12": "Lóan er komin",
    "s13": "Maístjarnan",
    "s14": "Ó blessuð vertu sumarsól",
    "s15": "Söknuður"
}
num_texts = len(texts)

for i in range(num_texts):
    filename = f"s{i+1}"
    with open(f"../data/search/{filename}.txt") as file:
        # search_lemmas[filename] = g.parse(file.read())
        current_lemmas = []
        for sent in g.parse(file.read())["sentences"]:
            if sent.lemmas is not None:
                for l in sent.lemmas:
                    current_lemmas.append(l)
        
        search_lemmas[filename] = current_lemmas

In [12]:
def search_texts(search_string = None):
    if search_string is None:
        return None

    # lemmum search strenginn
    search_string_lemmas = []
    for sent in g.parse(search_string)["sentences"]:
        if sent.lemmas is not None:
            for l in filter(lambda x: x not in stops, sent.lemmas):
                search_string_lemmas.append(l)

    # leitum nú í gegnum lemmurnur frá textunum og gefum þeim stig eftir því hversu oft þau koma fyrir
    search_results = {}
    output = dict()
    for filename, lemmas in search_lemmas.items():
        search_results[filename] = 0
        for lemma in lemmas:
            if lemma in search_string_lemmas:
                search_results[filename] += 1

    # prentum út niðurstöðurnar
    result = Counter(search_results)
    if result.total() == 0:
        print("Engar niðurstöður fundust")
    else:
        top_result = result.most_common(1)[0]
        with open(f"../data/search/{top_result[0]}.txt") as file:
            output["search"] =  search_string_lemmas
            output["top_hit"] = f"{texts[top_result[0]]}" 
            output["full_text"] =  file.read()
            return output
    


In [15]:
s = search_texts("stjörnurnar og sólin")
print("Leitarlemmur: ", s["search"])
print("Efsta match:   ", s["top_hit"])

Leitarlemmur:  ['stjarna', 'sól']
Efsta match:    Maístjarnan


In [16]:
s = search_texts("jól og kaka")
print("Leitarlemmur: ", s["search"])
print("Efsta match:   ", s["top_hit"])

Leitarlemmur:  ['jól', 'kaka']
Efsta match:    Snati og Óli


In [17]:
s = search_texts("lag um sofandi barn og nóttina")
print("Leitarlemmur: ", s["search"])
print("Efsta match:   ", s["top_hit"])

Leitarlemmur:  ['lag', 'sofa', 'barn', 'nótt']
Efsta match:    Sofðu unga ástin mín


## 4
### 1.)

Í kerfum til að þýða texta eða önnur form af máli er mjög mikilvægt að hafa réttar skilgreiningar útfrá upphaflega textanum. Það er yfirleitt ekki nóg að þýða einfaldlega orð fyrir orð. 
Þessar týpur af villum voru algengar í *google translate* þegar það var fyrst að koma fram í sviðsljósið. 
Þetta eru villur á borð við þýðinguna "Ég sá mann með sjónauka", það mætti túlka þessa setningu á tvo vegu, "ég sá mann og hann var með sjónauka" eða "ég var að nota sjónauka og sá þannig mann". 

Ef við pörsum textann áður en við þýðum hann getum við fengið þau orð sem eiga saman í hóp sem einfaldar að fá rétta þýðingu.

### 2.)
Fyrir einfalt sentiment analysis þar sem við höfum bara tvo flokka, vont og gott, myndi lemmun líklega ekki hamla okkur mikið.
Hinsvegar þegar við förum að bæta við fleiri og fleiri flokkum, geta leynst meiri og meiri upplýsingar innan beyginga og áherslu orða sem gæti horfið við lemmunina.

Fyrir keyword extraction getur verið gott að lemma fyrst, lemmunin kemur í veg fyrir að við höfum mörg samheiti fyrir sömu orðin til þess að fylgjast með, td get strengirnir *"áhugamálið mitt er hlaup"* og *"ég elska að vera á hlaupum"* verið tengdir báðir við orðið *"hlaupa"*

Fyrir auto paraphrasing erum við að fá sömu bónusa og fyrir keyword extraction. Þar sem að búið er að brjóta setningu niður í sína minnstu bita án þess að breyta meiningunni er auðvelt að bæta við orðum á milli eða breyta röð til þess að fá sömu meiningu án þess að nota alveg sömu orð.